In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from HFCs_hmimFAP_PR import configuration
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
from pyomo.opt import TerminationCondition, SolverStatus

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
data = pd.read_csv('R32_hmimFAP_10_2.csv')
data2 = pd.read_csv('R125_hmimFAP_10_2.csv')

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Make outlet arrays
IL_x = np.zeros(len(data["1"]))
IL_y = np.zeros(len(data["1"]))
R32_x = np.zeros(len(data["1"]))
R32_y = np.zeros(len(data["1"]))
R125_x = np.zeros(len(data["1"]))
R125_y = np.zeros(len(data["1"]))

alpha1 = np.zeros(len(data["1"]))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "hmimFAP"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)

for i in range(len(data["0"])):
    
    m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m1.fs.properties.PR_kappa_A['R32', 'hmimFAP'].fix(data["1"].iloc[i])
    m1.fs.properties.PR_kappa_A['hmimFAP', 'R32'].fix(data["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['hmimFAP', 'R125'].fix(data2["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['R125', 'hmimFAP'].fix(data2["1"].iloc[i])

    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "hmimFAP"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.2) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1)

    m1.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)

    try:
        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)

        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "hmimFAP"])
            IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "hmimFAP"])
            R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            print(i)
            
            alpha1[i] = (R125_y[i]/R125_x[i])/(R32_y[i]/R32_x[i])

    except ValueError:
        alpha1[i] = 'NaN'
       
        
results = pd.DataFrame(alpha1)
results.to_csv('hmimFAP_alpha_10_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
print(alpha1)

[1.03895485 0.84117315 1.0022159  0.69524251 1.07567769 0.97733503
 0.71427711 1.06205215 1.0474262  1.06718108 1.06514951 1.05714688
 1.05683645 1.06083315 1.05068559 1.06286807 1.0553851  1.0620843
 1.06545515 0.70777163 1.06041933 0.98448666 1.04271906 1.06317241
 1.06576009 1.00529718 1.0073391  1.06137997 1.06235857 0.9868905
 1.06264327 1.01139392 1.03519815 1.00802143 1.03321467 0.67495377
 1.06964729 1.07052394 1.02662533 1.00733943 0.94641555 1.06918084
 0.88375447 1.0434087  1.06734973 1.05663452 1.06338922 1.05635173
 1.06927857 1.07088301 1.0346831  1.01823061 1.01925084 1.06696396
 1.05586675 0.99106085 1.06893601 1.06952799 1.06642603 1.05279997
 0.99337492 1.0026162  1.06862196 1.03795661 1.06189738 0.60466526
 1.06285786 0.64677378 1.03625077 0.95909118 1.04624977 1.00683902
 0.79412566 1.05994342 1.06587737 1.06428085 0.95992433 0.97342033
 1.06732993 1.06580844 1.01756128 1.0907982  1.06411016 1.07022258
 1.06503529 0.60388986 1.01145432 0.99144925 0.97947794 0.985950